In [ ]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [ ]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

```json
User:
Create a travel plan for a family of 4, with 2 kids, from Singapore to Melboune
Validated Travel Plan:
{
    "main_task": "Plan a family trip from Singapore to Melbourne for 4 people including 2 kids.",
    "subtasks": [
        {
            "assigned_agent": "FlightBooking",
            "task_details": "Book round-trip flights from Singapore to Melbourne for 2 adults and 2 children."
        },
        {
            "assigned_agent": "HotelBooking",
            "task_details": "Find and book a family-friendly hotel in Melbourne that accommodates 4 people."
        },
        {
            "assigned_agent": "CarRental",
            "task_details": "Arrange for a car rental in Melbourne suitable for a family of 4."
        },
        {
            "assigned_agent": "ActivitiesBooking",
            "task_details": "Plan and book family-friendly activities in Melbourne suitable for kids."
        },
        {
            "assigned_agent": "DestinationInfo",
            "task_details": "Provide information about Melbourne, including attractions, dining options, and family-oriented activities."
        }
    ]
}
```

## গুরুত্বপূর্ণ তথ্য

এই ডকুমেন্টটি আপনাকে একটি নির্দিষ্ট বিষয়ে নির্দেশনা প্রদান করবে। দয়া করে নিচের ধাপগুলো অনুসরণ করুন এবং নিশ্চিত করুন যে আপনি প্রতিটি ধাপ সঠিকভাবে সম্পন্ন করেছেন।

### প্রয়োজনীয়তা

- একটি বৈধ অ্যাকাউন্ট থাকা আবশ্যক।
- @@INLINE_CODE_0@@ ইনস্টল করা থাকতে হবে।
- ইন্টারনেট সংযোগ সক্রিয় থাকতে হবে।

### ধাপ ১: লগইন করুন

আপনার অ্যাকাউন্টে লগইন করতে, নিচের নির্দেশনা অনুসরণ করুন:

1. @@INLINE_CODE_1@@ খুলুন।
2. আপনার ব্যবহারকারীর নাম এবং পাসওয়ার্ড প্রবেশ করান।
3. "লগইন" বোতামে ক্লিক করুন।

[!NOTE] যদি আপনি আপনার পাসওয়ার্ড ভুলে যান, @@INLINE_CODE_2@@ ব্যবহার করে এটি পুনরুদ্ধার করুন।

### ধাপ ২: কনফিগারেশন আপডেট করুন

আপনার সিস্টেম কনফিগারেশন আপডেট করতে:

1. @@INLINE_CODE_3@@ ফাইলটি খুলুন।
2. নিচের কোডটি যোগ করুন:

   @@CODE_BLOCK_1@@

3. ফাইলটি সংরক্ষণ করুন এবং বন্ধ করুন।

[!WARNING] নিশ্চিত করুন যে আপনি সঠিক ফাইলটি সম্পাদনা করছেন। ভুল ফাইল সম্পাদনা করলে সিস্টেমে সমস্যা হতে পারে।

### ধাপ ৩: সিস্টেম পুনরায় চালু করুন

পরিবর্তন কার্যকর করতে, আপনার সিস্টেম পুনরায় চালু করুন। এটি করতে:

1. @@INLINE_CODE_4@@ চালান।
2. পুনরায় চালু হওয়ার জন্য অপেক্ষা করুন।

[!TIP] পুনরায় চালু করার আগে আপনার কাজ সংরক্ষণ করুন।

### সাধারণ সমস্যা ও সমাধান

#### সমস্যা: লগইন করতে পারছেন না

**সম্ভাব্য কারণ:** ভুল ব্যবহারকারীর নাম বা পাসওয়ার্ড।

**সমাধান:** 
- নিশ্চিত করুন যে আপনি সঠিক তথ্য প্রবেশ করছেন।
- পাসওয়ার্ড পুনরুদ্ধার করতে @@INLINE_CODE_2@@ ব্যবহার করুন।

#### সমস্যা: কনফিগারেশন পরিবর্তন কার্যকর হচ্ছে না

**সম্ভাব্য কারণ:** ফাইলটি সঠিকভাবে সংরক্ষণ করা হয়নি।

**সমাধান:** 
- @@INLINE_CODE_3@@ ফাইলটি পুনরায় খুলুন এবং নিশ্চিত করুন যে পরিবর্তনগুলি সঠিকভাবে সংরক্ষণ করা হয়েছে।
- সিস্টেম পুনরায় চালু করুন।

[!IMPORTANT] এই নির্দেশনাগুলি অনুসরণ করার সময় সতর্ক থাকুন। ভুল পদক্ষেপ সিস্টেমে সমস্যা সৃষ্টি করতে পারে।

### উপসংহার

এই ডকুমেন্টে বর্ণিত ধাপগুলি অনুসরণ করে আপনি সফলভাবে আপনার সিস্টেম কনফিগার করতে পারবেন। যদি আপনার আরও সাহায্যের প্রয়োজন হয়, দয়া করে আমাদের সহায়তা কেন্দ্রে যোগাযোগ করুন।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
